## Collect data

In [1]:
import polars as pl
from great_tables import GT, style, loc, nanoplot_options
import yfinance as yf


input_filename = "tsm.csv"


def _collect_data(filename: str) -> None:
    tsm = yf.download(
        "TSM", start="2019-01-01", end="2023-12-31", interval="1mo"
    )
    tsm.to_csv(filename)


_collect_data(input_filename)

[*********************100%***********************]  1 of 1 completed


## Generate df

In [2]:
(
    pl.read_csv(input_filename)
    .with_columns(
        pl.col("Date").cast(pl.Date),
        pl.col("Close").alias("lines"),
    )
    .group_by_dynamic("Date", every="1y")
    .agg("lines")
)

Date,lines
date,list[f64]
2019-01-01,"[37.619999, 39.049999, … 58.099998]"
2020-01-01,"[53.939999, 53.84, … 109.040001]"
2021-01-01,"[121.519997, 125.940002, … 120.309998]"
2022-01-01,"[122.629997, 107.010002, … 74.489998]"
2023-01-01,"[92.730003, 87.07, … 104.0]"


In [3]:
month_cols = [f"{month_index:02}" for month_index in range(1, 13)]


def tweak_df(filename: str) -> pl.DataFrame:
    return (
        pl.read_csv(filename)
        .with_columns(
            pl.col("Date").cast(pl.Date),
            pl.col("Close").alias("lines"),
        )
        .group_by_dynamic("Date", every="1y")
        .agg("lines")
        .with_columns(
            pl.col("Date").dt.year().alias("year"),
            *[
                pl.col("lines").list.get(idx).alias(month)
                for idx, month in enumerate(month_cols)
            ],
            pl.col("lines").alias("bars"),
        )
        .select("year", *month_cols, "lines", "bars")
    )


df = tweak_df(input_filename)
df

year,01,02,03,04,05,06,07,08,09,10,11,12,lines,bars
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],list[f64]
2019,37.619999,39.049999,40.959999,43.82,38.349998,39.169998,42.630001,42.630001,46.48,51.630001,53.09,58.099998,"[37.619999, 39.049999, … 58.099998]","[37.619999, 39.049999, … 58.099998]"
2020,53.939999,53.84,47.790001,53.130001,50.330002,56.77,78.889999,79.25,81.07,83.870003,97.019997,109.040001,"[53.939999, 53.84, … 109.040001]","[53.939999, 53.84, … 109.040001]"
2021,121.519997,125.940002,118.279999,116.739998,117.360001,120.160004,116.639999,119.010002,111.650002,113.699997,117.150002,120.309998,"[121.519997, 125.940002, … 120.309998]","[121.519997, 125.940002, … 120.309998]"
2022,122.629997,107.010002,104.260002,92.93,95.300003,81.75,88.480003,83.349998,68.559998,61.549999,82.980003,74.489998,"[122.629997, 107.010002, … 74.489998]","[122.629997, 107.010002, … 74.489998]"
2023,92.730003,87.07,93.019997,84.300003,98.589996,100.919998,99.150002,93.57,86.900002,86.309998,97.309998,104.0,"[92.730003, 87.07, … 104.0]","[92.730003, 87.07, … 104.0]"


## Make table

In [4]:
def make_gt(df: pl.DataFrame) -> GT:
    domain_min = df.select(month_cols).min().min_horizontal().item()  # 37.61
    domain_max = df.select(month_cols).max().max_horizontal().item()  # 125.94
    return (
        GT(df)
        .tab_header("TSM Stock Price", "2019 ~ 2023")
        .tab_spanner(label="Month", columns=month_cols)
        .tab_spanner(label="Trend", columns=["lines", "bars"])
        .tab_options(table_background_color="#F1F1F1")
        .data_color(
            columns=month_cols,
            palette=["#F5EFE7", "#E7CE91"],
            domain=[domain_min * 0.99, domain_max * 1.01],
        )
        .tab_style(
            style=[style.text(color="red"), style.text(weight="Bold")],
            locations=[
                loc.body(columns="02", rows=2),
                loc.body(columns="01", rows=0),
            ],
        )
        .fmt_currency(month_cols)
        .opt_all_caps()
        .fmt_nanoplot(
            "lines",
            plot_type="line",
            reference_line="mean",
            options=nanoplot_options(
                reference_line_color="black",
                show_reference_line=True,
                currency="USD",
            ),
        )
        .fmt_nanoplot(
            "bars",
            plot_type="bar",
            autoscale=True,
            options=nanoplot_options(currency="USD"),
        )
    )


gtbl = make_gt(df)
gtbl

GT(_tbl_data=shape: (5, 15)
┌──────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ year ┆ 01         ┆ 02         ┆ 03         ┆ … ┆ 11         ┆ 12        ┆ lines     ┆ bars      │
│ ---  ┆ ---        ┆ ---        ┆ ---        ┆   ┆ ---        ┆ ---       ┆ ---       ┆ ---       │
│ i32  ┆ f64        ┆ f64        ┆ f64        ┆   ┆ f64        ┆ f64       ┆ list[f64] ┆ list[f64] │
╞══════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 2019 ┆ 37.619999  ┆ 39.049999  ┆ 40.959999  ┆ … ┆ 53.09      ┆ 58.099998 ┆ [37.61999 ┆ [37.61999 │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ 9, 39.049 ┆ 9, 39.049 │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ 999, …    ┆ 999, …    │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ 58.09…    ┆ 58.09…    │
│ 2020 ┆ 53.939999  ┆ 53.84      ┆ 47.790001  ┆ … ┆ 97.019997  ┆ 109.04000 ┆ [53.93999 ┆ [53.93999 │
│      ┆            ┆            ┆            ┆   ┆            ┆ 1         ┆ 9, 53.84, ┆ 9, 53.84, │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ … 109.040 ┆ … 109.040 │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ 00…       ┆ 00…       │
│ 2021 ┆ 121.519997 ┆ 125.940002 ┆ 118.279999 ┆ … ┆ 117.150002 ┆ 120.30999 ┆ [121.5199 ┆ [121.5199 │
│      ┆            ┆            ┆            ┆   ┆            ┆ 8         ┆ 97, 125.9 ┆ 97, 125.9 │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ 40002, …  ┆ 40002, …  │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ 120…      ┆ 120…      │
│ 2022 ┆ 122.629997 ┆ 107.010002 ┆ 104.260002 ┆ … ┆ 82.980003  ┆ 74.489998 ┆ [122.6299 ┆ [122.6299 │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ 97, 107.0 ┆ 97, 107.0 │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ 10002, …  ┆ 10002, …  │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ 74.…      ┆ 74.…      │
│ 2023 ┆ 92.730003  ┆ 87.07      ┆ 93.019997  ┆ … ┆ 97.309998  ┆ 104.0     ┆ [92.73000 ┆ [92.73000 │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ 3, 87.07, ┆ 3, 87.07, │
│      ┆            ┆            ┆            ┆   ┆            ┆           ┆ … 104.0]  ┆ … 104.0]  │
└──────┴────────────┴────────────┴────────────┴───┴────────────┴───────────┴───────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x000001D59446F950>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.default: 1>, column_label='year', column_align='right', column_width=None), ColInfo(var='01', type=<ColInfoTypeEnum.default: 1>, column_label='01', column_align='right', column_width=None), ColInfo(var='02', type=<ColInfoTypeEnum.default: 1>, column_label='02', column_align='right', column_width=None), ColInfo(var='03', type=<ColInfoTypeEnum.default: 1>, column_label='03', column_align='right', column_width=None), ColInfo(var='04', type=<ColInfoTypeEnum.default: 1>, column_label='04', column_align='right', column_width=None), ColInfo(var='05', type=<ColInfoTypeEnum.default: 1>, column_label='05', column_align='right', column_width=None), ColInfo(var='06', type=<ColInfoTypeEnum.default: 1>, column_label='06', column_align='right', column_width=None), ColInfo(var='07', type=<ColInfoTypeEnum.default: 1>, column_label='07', column_align='right', column_width=None), ColInfo(var='08', type=<ColInfoTypeEnum.default: 1>, column_label='08', column_align='right', column_width=None), ColInfo(var='09', type=<ColInfoTypeEnum.default: 1>, column_label='09', column_align='right', column_width=None), ColInfo(var='10', type=<ColInfoTypeEnum.default: 1>, column_label='10', column_align='right', column_width=None), ColInfo(var='11', type=<ColInfoTypeEnum.default: 1>, column_label='11', column_align='right', column_widt

## Output HTML

In [5]:
output_filename = "tsm.html"


def _write_html(gtbl: GT, filename: str) -> None:
    with open(filename, "w") as f:
        f.write(gtbl.as_raw_html())


_write_html(gtbl, output_filename)